# Sorting NumPy Arrays

In [2]:
# Imports required but not shown in the video lecture.
import numpy

from numpy import argsort, array, linspace, searchsorted, sort
from numpy.random import rand

In [ ]:
print numpy.__version__

This workbook looks at the various sorting methods form NumPy arrays.  We'll concentrate on sorting numeric (or scalar arrays) and leave structured arrays to a separate discussion. There are several sorting tools to be aware of.  The primary functions to be aware of are [`sort`](http://docs.scipy.org/doc/numpy/reference/generated/numpy.sort.html#numpy.sort), [`argsort`](http://docs.scipy.org/doc/numpy/reference/generated/numpy.argsort.html#numpy.argsort), and [`searchsorted`](http://docs.scipy.org/doc/numpy/reference/generated/numpy.searchsorted.html#numpy.searchsorted).  Their are also method versions of each of these on NumPy arrays.  The [`sort`](http://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.sort.html?highlight=sort#numpy.ndarray.sort) method is useful for inplace sorting of array values.

Briefly, `sort` is useful for ordering the values of an array, `argsort` is useful for ordering the indices that would make an arrays values sorted, and `searchsorted` is used to find where values in one array would fall within another sorted array.  We'll show examples of each. 

## sort and argsort

In [ ]:
# We'll start with a couple of related arrays.  The first has the 
# names of several children, and the second has their weights.
names = array(['bob', 'sue', 'jan', 'ed'])
weights = array([ 20.8,  93.2,  53.4, 61.8])

In [ ]:
# Return a new array with weights in order.
sort(weights)

In [ ]:
# Use argsort to get the sorted indices order.
ordered_indices = argsort(weights)
ordered_indices

In [ ]:
# You can use fancy indexing to access the values of a in order.
weights[ordered_indices]

In [ ]:
# And, you can use ordered_indices to order the children's names 
# in ascending order of weight.  This is a *very* useful feature.
names[ordered_indices]

In [ ]:
# Sort a 'inplace' using the sort method
a = array([2.3, 1.2, 4.5, 3.1])
a.sort()
a

In [3]:
# Let's look at a 2D array.  By default each of the rows is independently sorted.
a = array([[.2, .1, .5],
           [.4, .8, .3],
           [.9, .6, .7]])
sort(a)

array([[ 0.1,  0.2,  0.5],
       [ 0.3,  0.4,  0.8],
       [ 0.6,  0.7,  0.9]])

In [4]:
# Or, you can have the columns independently sorted.
sort(a, axis=0)

array([[ 0.2,  0.1,  0.3],
       [ 0.4,  0.6,  0.5],
       [ 0.9,  0.8,  0.7]])

In [ ]:
# argsort has the same behavior.  Here each column the sorted indices in it.
argsort(a, axis=0)

## searchsorted

`searchsorted(sorted, values)` is used for finding where items in values would fall in the `sorted` array.  Although, it is not the fastest approach, it could be used for histogramming. 

In [5]:
# Create an ordered list of numbers.
sorted = linspace(0,1,5)
sorted

array([ 0.  ,  0.25,  0.5 ,  0.75,  1.  ])

In [6]:
values = array([.1, .8, .3, .12, .5, .25])
# Now find which index each number in values would inserted before in the 
# bins array.  Note that numbers equivalent to an entry in bins are inserted 
# before that number.  In this case, .25 is inserted at index 1.
searchsorted(sorted, values)

array([1, 4, 2, 1, 2, 1])

One good use case for this is, given a sorted set of numbers, imagine you want to find out the indices that demark values within a certain range.  You could do this with the following.

In [7]:
data = rand(100)
data.sort()
bounds = .4, .6
low_index, high_index = searchsorted(data, bounds)
# This should extract the values between .4 and .6
data[low_index:high_index]

array([ 0.40365314,  0.40520276,  0.40993364,  0.41572747,  0.42136447,
        0.42167157,  0.42367224,  0.43008579,  0.43020542,  0.43185927,
        0.43613509,  0.45704605,  0.46584321,  0.47506117,  0.48133056,
        0.48172586,  0.48230786,  0.49909364,  0.51126601,  0.53091904,
        0.53164955,  0.53183286,  0.5513743 ,  0.55839878,  0.56576759,
        0.56850132])

## Sorting speed for 2D arrays

When sorting a 2D array, you are always sorting along a particular axis.  Here we explore the sorting of a 1000x1000 array of random floating point values.  This section explores the trade-offs between data layout and sorting algorithm choice in terms of speed.

The syntax of NumPy's sorting method is as follows:

            sort(a, axis=-1, kind='quicksort', order=None)

Here `a` is a potentially multi-dimensional array to sort, `axis` specifies along which axis of the array values will be sorted, and `kind` defines the sorting algorithm used.  `order` only applies to structured arrays.

For `kind` the following options are available, comparing their relative speed (lower is better), memory usage, and [stability](http://en.wikipedia.org/wiki/Sorting_algorithm#Stability):

<table border="1" class="docutils">
<colgroup>
<col width="22%" />
<col width="14%" />
<col width="26%" />
<col width="24%" />
<col width="14%" />
</colgroup>
<thead valign="bottom">
<tr class="row-odd"><th class="head">kind</th>
<th class="head">speed</th>
<th class="head">worst case</th>
<th class="head">work space</th>
<th class="head">stable</th>
</tr>
</thead>
<tbody valign="top">
<tr class="row-even"><td>&#8216;quicksort&#8217;</td>
<td>1</td>
<td>O(n^2)</td>
<td>0</td>
<td>no</td>
</tr>
<tr class="row-odd"><td>&#8216;mergesort&#8217;</td>
<td>2</td>
<td>O(n*log(n))</td>
<td>~n/2</td>
<td>yes</td>
</tr>
<tr class="row-even"><td>&#8216;heapsort&#8217;</td>
<td>3</td>
<td>O(n*log(n))</td>
<td>0</td>
<td>no</td>
</tr>
</tbody>
</table>



In [8]:
a = rand(1000,1000)

In [9]:
%precision 3
a

array([[ 0.282,  0.366,  0.918, ...,  0.517,  0.767,  0.572],
       [ 0.889,  0.331,  0.683, ...,  0.009,  0.255,  0.17 ],
       [ 0.085,  0.172,  0.095, ...,  0.91 ,  0.693,  0.727],
       ..., 
       [ 0.644,  0.314,  0.134, ...,  0.24 ,  0.89 ,  0.633],
       [ 0.022,  0.279,  0.362, ...,  0.884,  0.925,  0.115],
       [ 0.516,  0.842,  0.181, ...,  0.654,  0.446,  0.369]])

#### Speed of Sorting Along Different Axes

Compare the speed of sorting along the last axis (contiguous memory) and the first axis where you are sorting non-contiguous memory.  In the latter case, NumPy makes a copy of the data before it can do the sort.  For a 1000x1000 array, sorting along the last axis is typically about 3x faster.  Your milage will vary depending on your array size.

In [10]:
timeit sort(a, axis=-1)

10 loops, best of 3: 40.4 ms per loop


In [11]:
timeit sort(a, axis=0)

10 loops, best of 3: 43.9 ms per loop


#### Sorting Random Data: Comparing sort types

Comparing quicksort, mergesort, and heapsort on random data.

In [12]:
timeit sort(a, kind='quicksort', axis=-1)

10 loops, best of 3: 40.2 ms per loop


In [13]:
timeit sort(a, kind='mergesort', axis=-1)

10 loops, best of 3: 45.7 ms per loop


In [14]:
timeit sort(a, kind='heapsort', axis=-1)

10 loops, best of 3: 70.3 ms per loop


#### Comparing sort of already sorted data

In [15]:
# in place sort of a so that it is now ordered along the last axis.
a.sort() 

In [16]:
timeit sort(a, kind='quicksort', axis=-1)

100 loops, best of 3: 7.3 ms per loop


In [17]:
timeit sort(a, kind='mergesort', axis=-1)

100 loops, best of 3: 9.33 ms per loop


In [18]:
timeit sort(a, kind='heapsort', axis=-1)

10 loops, best of 3: 54.1 ms per loop


Copyright 2008-2016, Enthought, Inc.<br>Use only permitted under license.  Copying, sharing, redistributing or other unauthorized use strictly prohibited.<br>http://www.enthought.com